In [1]:
#hide
# !pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [2]:
#hide
from fastbook import *
from fastai.vision.widgets import *

In [3]:
path = Path()
learn_inf = load_learner(path/'export.pkl', cpu=True)
btn_upload = widgets.FileUpload()
out_pl = widgets.Output()
lbl_pred = widgets.Label()

In [9]:
# test def on_click_classify(change):
def on_click(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred,pred_idx,probs = learn_inf.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

#btn_run.on_click(on_click_classify)

In [6]:
#hide
#Putting back btn_upload to a widget for next cell
# test btn_upload = widgets.FileUpload()

In [10]:
btn_upload.observe(on_click, names=['data'])

In [12]:
#hide_output
display(VBox([widgets.Label('Selecione seu urso!'), 
      btn_upload, out_pl, lbl_pred]))
#but_run